# Lesson 10: CNN
https://www.youtube.com/watch?v=LgFNRIFxuUo&list=PLlMkM4tgfjnJ3I-dbhO9JTw7gNty6o_2m&index=10

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [22]:
torch.manual_seed(1249583)

Define `Dataset`

In [23]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=10, shuffle=True)

In [24]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1, shuffle=True)

In [30]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# 1. Model

In [26]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=1) # in_channel is 1. Because it has 1 color
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=1)
        
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [27]:
model = Net()

# 2. Loss & Optimizer
criterion has been absorbed to the `train and test` functions already

In [28]:
optimizer = optim.SGD(model.parameters(), lr=0.001 , momentum=0.1)

# 3. Train

In [29]:
for epoch in range(1, 1 + 1):
    train(model, 'cpu', train_loader, optimizer, epoch)
    test(model, 'cpu', test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.244570
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.235052
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.192628
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.079259
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.862965
Train Epoch: 1 [5000/60000 (8%)]	Loss: 1.794037
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.414833
Train Epoch: 1 [7000/60000 (12%)]	Loss: 1.080190
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.758389
Train Epoch: 1 [9000/60000 (15%)]	Loss: 0.644825
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.503210
Train Epoch: 1 [11000/60000 (18%)]	Loss: 1.205046
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.486211
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.526567
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.531859
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.606589
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.183819
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.252317
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.688382
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.457156
Train Epoch: 1 [200